In [189]:
import torch
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import dataset

from torch import Tensor, nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer



In [190]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


device = try_gpu(0)

# Preprocessing Text

In [191]:
import regex as re

with open('../data/ne_dedup.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'\s*[\u0964]\s*', r'\u0020\u0964\u0020', text)
    text = re.sub(r'\s*[\u003f]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*[\u002c]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*\n\s*','\n', text)
    #text = re.sub(r'\s*[\u0966-\u0976]+\s*','\u0020[\u0966-\u0976]\u0020', text)
    #text = re.sub(r'\s+?\s+', r'\u0020?\u0020', text)
    text = re.sub(r'[^\u0900-\u097F,?\s+]','', text)
    
    

In [192]:
c = re.sub(r'[^\u0900-\u097F,?]','','?नमस्तेnishant,')
#c = re.sub(r'\s*[\u002c]\s*','\u0020\u002c\u0020','?नमस्तेnishant,')
c


'?नमस्ते,'

In [193]:
hex(ord('०'))

'0x966'

In [194]:
len(text.split('\n'))

341961

In [195]:
train_iter_first = text.split('\n')[:30000]
train_iter_second = text.split('\n')[:35000]
test_iter = text.split('\n')[35000:40000]

In [196]:
train_iter_first[:100]

['बर्दिबास नगरपालिकाको तेस्रो नगर परिषदबाट पारित आव२०७३ । ७४ को संशोधित र २०७४ । ७५ को प्रस्तावित नीति ? कार्यक्रम तथा बजेट',
 'अार्थिक वर्ष २०७५७६ काे नदिजन्य पदार्थकाे उत्खनन् गरी बिक्रि वितरण तथा अान्तरिक निकासी गर्ने कार्यकाे बाेलपत्र सम्बन्धी सुचना',
 'सक्षार सप्तरी अभियानमा सप्तरीबासी सम्पूर्ण सरोकारवालाहरुको सहयोग र सहभागिताकाो लागि अनुराोध छ ।  सामुदायिक अध्ययन केन्द्रहरूको नविकरण सम्बन्धमा । ',
 'काठमाडौं ? १२ कातिक । राष्ट्रपति विद्यादेवी भण्डारी मित्रराष्ट्र कतारको चार दिवसीय औपचारिक भ्रमणमा आज त्यसतर्फ प्रस्थान गरेकी छन् । राष्ट्रपति विद्यादेवी भण्डारी कतारका अमिर शेख हमाद बीन खालिदा अल थानीको मैत्रीपूर्ण निमन्त्रणामा चार दिवसीय औपचारिक',
 'काठमाडौँ ? २६ कात्तिक । सरकारले सङ्घ ? प्रदेश र स्थानीय तहमा कर्मचारी समायोजन गर्नका लागि कर्मचारी समायोजन अध्यादेश२०७५ ल्याउने तयारी गरेको छ । सरकारले यसअघि ल्याएको',
 'काठमाडौं ? २६ कातिक । महानायक राजेश हमाल अहिले चलचित्र क्षेत्रमा पातलिए पनि उनको सिने जगतमा नामै काफी छ । कुनै समय बलिउड सुपरस्टार अमिताभ वच्चनसँग',
 'काठमाडौं ? २६ काति

In [197]:
print(len(train_iter_first),len(train_iter_second),len(test_iter))

30000 35000 5000


In [198]:
tokenizer = get_tokenizer(None)
vocab = build_vocab_from_iterator(
    map(tokenizer, train_iter_first), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])


dummy_vocab = build_vocab_from_iterator(
    map(tokenizer, train_iter_second), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])
print(len(dummy_vocab))

377990


In [199]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long)
            for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [200]:
#device = torch.device('cpu')

In [201]:
#Train and Test Split

train_data = data_process(train_iter_second)
test_data = data_process(test_iter)

In [202]:
train_data[:100]

tensor([ 19020,   1025,    511,    761,  24762,    827, 103693,      1,   3726,
            32,  11495,      3,    792,      1,   1374,     32,   2702,    324,
             2,    107,     14,    330,  24336,     77,   5527,   8387,  28118,
        123255,  11037,     33,   6619,    480,     14,  34020,   3207,     13,
        176772,  44337,    640,   2987, 314958,   4492,   1852, 316978,    436,
         20999,    130,      3, 321145,      9, 151169,      4,      1,    950,
           305,  67904,  10488,    693,      1,     60,      2,    203,  27756,
             1,    387,   3505,   1589,  15993,   9954,    230,  11080,   1190,
          1417,     98,   6243,   3793,    783,      7,      1,    387,   3505,
          1589,  14473,  16876,   7713,  33868,  40783,  32454,   4344,  47676,
          7188,   9612,    230,  11080,   1190,    161,      2,    593,    302,
             1])

In [203]:
print(len(vocab))
print(len(train_iter_second),len(test_iter))
torch.cuda.empty_cache() 

torch.cuda.memory_allocated() 

342570
35000 5000


2531019776

In [204]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [205]:
# batch_size = 35
# eval_batch_size = 10

# train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# test_data = batchify(test_data, eval_batch_size)

# Working with a dummy Sample

In [206]:
#Sample Data


text = ['आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य']
#text = ['जनसंख्या']
sample_data = data_process(
    text)

In [207]:
sample_data

tensor([ 2086,  5694,   568,     0,   897, 28361,     0,   357,   465,   410,
         6548,   293,     0,   357,   465])

In [208]:
sample_data = batchify(sample_data, 2)
print("Given word:", text[0])
sample_data

Given word: आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य


tensor([[ 2086,   357],
        [ 5694,   465],
        [  568,   410],
        [    0,  6548],
        [  897,   293],
        [28361,     0],
        [    0,   357]], device='cuda:0')

In [209]:
train_data.shape

torch.Size([7540440])

In [210]:
bptt = 16
train_data = batchify(train_data, bptt)  # shape [seq_len, batch_size]
test_data = batchify(test_data, bptt)


import math
def get_batch(source: Tensor, i: int) -> tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    #target = source[i+1:i+1+seq_len]
    return data, target

# Model Definition

In [211]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)





class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(
            d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [212]:
# torch.cuda.memory_stats()


In [213]:
torch.cuda.memory_allocated()

2599095296

In [214]:
#device = torch.device('cpu')

# Hyper-Parameter Tuning

In [215]:
ntokens = len(vocab)  # size of vocabulary
emsize = 300  # embedding dimension
d_hid = 800  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # number of heads in nn.MultiheadAttention
dropout = 0.05  # dropout probability
model = TransformerModel(ntokens, emsize,nhead, d_hid,
                         nlayers, dropout).to(device)

In [216]:
criterion = nn.CrossEntropyLoss()
lr = 1  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [217]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [218]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        #print(type(output))
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [485]:
softmax = nn.Softmax(dim=2)
# softmax = nn.LogSoftmax(dim=2)

In [220]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_softmax = softmax(output)
            output_softmax_permuted = output_softmax.permute(1, 0, 2)
            indices = torch.argmax(output_softmax_permuted, dim=2)
            target_indices = targets.t()
            # print(output)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()

    return total_loss / (len(eval_data) - 1)


In [221]:
#            print('data')
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in data.t()]))
#             print(indices)
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in indices]))
#             print(len(targets))
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in target_indices]))

In [222]:
        # target_indices = targets.t()
        # print(indices.shape)
        # print(indices)
        # temp_gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0][i]
        # gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0]
        # print(temp_text)
        # print([[vocab.lookup_tokens(list(index))
        #        for index in indices][0][i]])
#         temp_text = [[vocab.lookup_tokens(list(index))]
#                       for index in indices[0][i]]
        # print(temp_text)
#         temp_text = [' '.join(temp_text)]
        # print(temp_text)
        # gen_data = vocab.lookup_tokens((list(gen_data))) + list(temp_gen_data)
        # gen_data = torch.tensor(
        #    (list(gen_data[0])+list(indices[0][i]))).unsqueeze(0)
        # gen_data = torch.concat([gen_data, torch.tensor(indices[0][i])], dim=1)
        #gen_data = ' '.join(gen_data)
        #return temp_txt
#         gen_data = data_process(temp_text)
#         gen_data = batchify(gen_data, 1)
        #return gen_data

In [223]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Training Data

In [81]:
# Loop over epochs. Save the model if the validation loss is the best
# we've seen so far. Adjust the learning rate after each epoch.
import time
import copy
best_val_loss = float('inf')
epochs = 8
best_model = None
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    eval_loss = evaluate(model, test_data)
    eval_ppl = math.exp(eval_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
         f'valid loss {eval_loss:5.2f} | valid ppl {eval_ppl:8.2f}')
    print('-' * 89)
    #best_model = copy.deepcopy(model)
    if eval_loss < best_val_loss:
       best_val_loss = eval_loss
       best_model = copy.deepcopy(model)

#     scheduler.step()
#save model
#torch.save(best_model.state_dict(),'best_model_3bigx3.pt')
torch.save(best_model.state_dict(),'best_model_3bigx3_corrected.pt')
torch.save(model.state_dict(),'training_model_3bigx3_corrected.pt')


| epoch   1 |   200/29454 batches | lr 1.00 | ms/batch 153.47 | loss 10.72 | ppl 45202.67
| epoch   1 |   400/29454 batches | lr 1.00 | ms/batch 150.94 | loss  9.61 | ppl 14900.30
| epoch   1 |   600/29454 batches | lr 1.00 | ms/batch 151.98 | loss  9.26 | ppl 10511.59
| epoch   1 |   800/29454 batches | lr 1.00 | ms/batch 152.24 | loss  9.10 | ppl  8980.99
| epoch   1 |  1000/29454 batches | lr 1.00 | ms/batch 152.16 | loss  8.98 | ppl  7973.21
| epoch   1 |  1200/29454 batches | lr 1.00 | ms/batch 152.26 | loss  8.79 | ppl  6593.27
| epoch   1 |  1400/29454 batches | lr 1.00 | ms/batch 152.27 | loss  8.70 | ppl  5998.29
| epoch   1 |  1600/29454 batches | lr 1.00 | ms/batch 153.06 | loss  8.64 | ppl  5667.70
| epoch   1 |  1800/29454 batches | lr 1.00 | ms/batch 152.68 | loss  8.64 | ppl  5674.85
| epoch   1 |  2000/29454 batches | lr 1.00 | ms/batch 152.50 | loss  8.53 | ppl  5072.71
| epoch   1 |  2200/29454 batches | lr 1.00 | ms/batch 152.49 | loss  8.50 | ppl  4928.34
| epoch   

| epoch   1 | 18600/29454 batches | lr 1.00 | ms/batch 152.56 | loss  7.58 | ppl  1962.75
| epoch   1 | 18800/29454 batches | lr 1.00 | ms/batch 152.65 | loss  7.57 | ppl  1929.49
| epoch   1 | 19000/29454 batches | lr 1.00 | ms/batch 152.69 | loss  7.34 | ppl  1545.26
| epoch   1 | 19200/29454 batches | lr 1.00 | ms/batch 152.71 | loss  7.27 | ppl  1434.26
| epoch   1 | 19400/29454 batches | lr 1.00 | ms/batch 152.70 | loss  7.35 | ppl  1554.65
| epoch   1 | 19600/29454 batches | lr 1.00 | ms/batch 152.58 | loss  7.21 | ppl  1352.80
| epoch   1 | 19800/29454 batches | lr 1.00 | ms/batch 152.56 | loss  7.27 | ppl  1441.17
| epoch   1 | 20000/29454 batches | lr 1.00 | ms/batch 152.54 | loss  7.34 | ppl  1536.96
| epoch   1 | 20200/29454 batches | lr 1.00 | ms/batch 152.53 | loss  7.37 | ppl  1585.04
| epoch   1 | 20400/29454 batches | lr 1.00 | ms/batch 152.59 | loss  7.28 | ppl  1449.97
| epoch   1 | 20600/29454 batches | lr 1.00 | ms/batch 152.59 | loss  7.38 | ppl  1609.06
| epoch   

| epoch   2 |  7000/29454 batches | lr 1.00 | ms/batch 152.86 | loss  7.10 | ppl  1207.73
| epoch   2 |  7200/29454 batches | lr 1.00 | ms/batch 152.75 | loss  6.98 | ppl  1080.15
| epoch   2 |  7400/29454 batches | lr 1.00 | ms/batch 152.82 | loss  7.01 | ppl  1103.26
| epoch   2 |  7600/29454 batches | lr 1.00 | ms/batch 152.70 | loss  7.19 | ppl  1319.76
| epoch   2 |  7800/29454 batches | lr 1.00 | ms/batch 152.82 | loss  7.08 | ppl  1193.51
| epoch   2 |  8000/29454 batches | lr 1.00 | ms/batch 152.81 | loss  7.35 | ppl  1554.22
| epoch   2 |  8200/29454 batches | lr 1.00 | ms/batch 152.85 | loss  7.01 | ppl  1108.28
| epoch   2 |  8400/29454 batches | lr 1.00 | ms/batch 152.74 | loss  6.89 | ppl   983.90
| epoch   2 |  8600/29454 batches | lr 1.00 | ms/batch 152.86 | loss  7.10 | ppl  1206.01
| epoch   2 |  8800/29454 batches | lr 1.00 | ms/batch 152.78 | loss  7.05 | ppl  1151.95
| epoch   2 |  9000/29454 batches | lr 1.00 | ms/batch 152.84 | loss  6.83 | ppl   929.74
| epoch   

| epoch   2 | 25400/29454 batches | lr 1.00 | ms/batch 152.73 | loss  6.81 | ppl   905.35
| epoch   2 | 25600/29454 batches | lr 1.00 | ms/batch 152.77 | loss  6.81 | ppl   905.63
| epoch   2 | 25800/29454 batches | lr 1.00 | ms/batch 152.71 | loss  6.75 | ppl   855.11
| epoch   2 | 26000/29454 batches | lr 1.00 | ms/batch 152.77 | loss  6.90 | ppl   992.94
| epoch   2 | 26200/29454 batches | lr 1.00 | ms/batch 152.98 | loss  6.66 | ppl   784.22
| epoch   2 | 26400/29454 batches | lr 1.00 | ms/batch 152.82 | loss  6.82 | ppl   920.46
| epoch   2 | 26600/29454 batches | lr 1.00 | ms/batch 152.71 | loss  6.75 | ppl   852.40
| epoch   2 | 26800/29454 batches | lr 1.00 | ms/batch 152.73 | loss  6.94 | ppl  1028.38
| epoch   2 | 27000/29454 batches | lr 1.00 | ms/batch 152.63 | loss  6.90 | ppl   995.50
| epoch   2 | 27200/29454 batches | lr 1.00 | ms/batch 152.71 | loss  6.80 | ppl   894.98
| epoch   2 | 27400/29454 batches | lr 1.00 | ms/batch 152.77 | loss  6.80 | ppl   901.30
| epoch   

| epoch   3 | 13800/29454 batches | lr 1.00 | ms/batch 152.77 | loss  6.74 | ppl   843.76
| epoch   3 | 14000/29454 batches | lr 1.00 | ms/batch 152.71 | loss  6.63 | ppl   755.16
| epoch   3 | 14200/29454 batches | lr 1.00 | ms/batch 152.77 | loss  6.62 | ppl   752.67
| epoch   3 | 14400/29454 batches | lr 1.00 | ms/batch 152.67 | loss  6.74 | ppl   845.77
| epoch   3 | 14600/29454 batches | lr 1.00 | ms/batch 152.74 | loss  6.60 | ppl   733.52
| epoch   3 | 14800/29454 batches | lr 1.00 | ms/batch 152.67 | loss  6.49 | ppl   661.01
| epoch   3 | 15000/29454 batches | lr 1.00 | ms/batch 152.65 | loss  6.48 | ppl   652.73
| epoch   3 | 15200/29454 batches | lr 1.00 | ms/batch 152.81 | loss  6.42 | ppl   610.94
| epoch   3 | 15400/29454 batches | lr 1.00 | ms/batch 152.64 | loss  6.45 | ppl   633.30
| epoch   3 | 15600/29454 batches | lr 1.00 | ms/batch 152.70 | loss  6.45 | ppl   634.47
| epoch   3 | 15800/29454 batches | lr 1.00 | ms/batch 152.76 | loss  6.42 | ppl   611.99
| epoch   

...

| epoch   6 | 15800/29454 batches | lr 1.00 | ms/batch 152.94 | loss  5.85 | ppl   348.48
| epoch   6 | 16000/29454 batches | lr 1.00 | ms/batch 152.94 | loss  5.90 | ppl   366.75
| epoch   6 | 16200/29454 batches | lr 1.00 | ms/batch 152.95 | loss  5.93 | ppl   377.28
| epoch   6 | 16400/29454 batches | lr 1.00 | ms/batch 152.93 | loss  6.01 | ppl   407.54
| epoch   6 | 16600/29454 batches | lr 1.00 | ms/batch 152.93 | loss  5.87 | ppl   353.56
| epoch   6 | 16800/29454 batches | lr 1.00 | ms/batch 153.00 | loss  5.92 | ppl   373.95
| epoch   6 | 17000/29454 batches | lr 1.00 | ms/batch 153.00 | loss  5.93 | ppl   375.36
| epoch   6 | 17200/29454 batches | lr 1.00 | ms/batch 152.93 | loss  6.07 | ppl   434.14
| epoch   6 | 17400/29454 batches | lr 1.00 | ms/batch 152.98 | loss  5.84 | ppl   344.59
| epoch   6 | 17600/29454 batches | lr 1.00 | ms/batch 153.00 | loss  5.89 | ppl   360.13
| epoch   6 | 17800/29454 batches | lr 1.00 | ms/batch 152.95 | loss  6.06 | ppl   428.42
| epoch   

| epoch   7 |  4200/29454 batches | lr 1.00 | ms/batch 153.74 | loss  5.82 | ppl   337.34
| epoch   7 |  4400/29454 batches | lr 1.00 | ms/batch 152.92 | loss  5.79 | ppl   326.00
| epoch   7 |  4600/29454 batches | lr 1.00 | ms/batch 152.91 | loss  5.90 | ppl   364.31
| epoch   7 |  4800/29454 batches | lr 1.00 | ms/batch 152.92 | loss  5.82 | ppl   338.56
| epoch   7 |  5000/29454 batches | lr 1.00 | ms/batch 152.97 | loss  5.97 | ppl   391.07
| epoch   7 |  5200/29454 batches | lr 1.00 | ms/batch 152.97 | loss  5.89 | ppl   363.17
| epoch   7 |  5400/29454 batches | lr 1.00 | ms/batch 152.98 | loss  5.94 | ppl   381.70
| epoch   7 |  5600/29454 batches | lr 1.00 | ms/batch 152.97 | loss  5.66 | ppl   287.87
| epoch   7 |  5800/29454 batches | lr 1.00 | ms/batch 153.07 | loss  5.82 | ppl   338.03
| epoch   7 |  6000/29454 batches | lr 1.00 | ms/batch 153.00 | loss  6.07 | ppl   434.81
| epoch   7 |  6200/29454 batches | lr 1.00 | ms/batch 152.98 | loss  5.89 | ppl   360.33
| epoch   

| epoch   7 | 22600/29454 batches | lr 1.00 | ms/batch 152.88 | loss  5.87 | ppl   354.24
| epoch   7 | 22800/29454 batches | lr 1.00 | ms/batch 152.93 | loss  5.79 | ppl   328.57
| epoch   7 | 23000/29454 batches | lr 1.00 | ms/batch 152.84 | loss  5.90 | ppl   364.60
| epoch   7 | 23200/29454 batches | lr 1.00 | ms/batch 152.87 | loss  5.84 | ppl   343.07
| epoch   7 | 23400/29454 batches | lr 1.00 | ms/batch 152.89 | loss  5.75 | ppl   314.05
| epoch   7 | 23600/29454 batches | lr 1.00 | ms/batch 152.92 | loss  5.90 | ppl   365.63
| epoch   7 | 23800/29454 batches | lr 1.00 | ms/batch 153.13 | loss  5.81 | ppl   334.38
| epoch   7 | 24000/29454 batches | lr 1.00 | ms/batch 152.99 | loss  5.83 | ppl   339.18
| epoch   7 | 24200/29454 batches | lr 1.00 | ms/batch 153.04 | loss  5.60 | ppl   270.06
| epoch   7 | 24400/29454 batches | lr 1.00 | ms/batch 152.94 | loss  5.67 | ppl   290.14
| epoch   7 | 24600/29454 batches | lr 1.00 | ms/batch 152.98 | loss  5.81 | ppl   335.21
| epoch   

| epoch   8 | 11000/29454 batches | lr 1.00 | ms/batch 152.96 | loss  5.59 | ppl   266.67
| epoch   8 | 11200/29454 batches | lr 1.00 | ms/batch 152.99 | loss  5.63 | ppl   278.16
| epoch   8 | 11400/29454 batches | lr 1.00 | ms/batch 152.97 | loss  5.58 | ppl   263.84
| epoch   8 | 11600/29454 batches | lr 1.00 | ms/batch 153.02 | loss  5.55 | ppl   257.05
| epoch   8 | 11800/29454 batches | lr 1.00 | ms/batch 153.00 | loss  5.60 | ppl   270.91
| epoch   8 | 12000/29454 batches | lr 1.00 | ms/batch 152.96 | loss  5.79 | ppl   325.69
| epoch   8 | 12200/29454 batches | lr 1.00 | ms/batch 152.98 | loss  5.80 | ppl   331.34
| epoch   8 | 12400/29454 batches | lr 1.00 | ms/batch 153.06 | loss  5.79 | ppl   327.78
| epoch   8 | 12600/29454 batches | lr 1.00 | ms/batch 153.14 | loss  5.55 | ppl   258.40
| epoch   8 | 12800/29454 batches | lr 1.00 | ms/batch 153.23 | loss  5.67 | ppl   290.62
| epoch   8 | 13000/29454 batches | lr 1.00 | ms/batch 153.09 | loss  5.57 | ppl   262.38
| epoch   

| epoch   8 | 29400/29454 batches | lr 1.00 | ms/batch 152.94 | loss  5.57 | ppl   263.21
-----------------------------------------------------------------------------------------
| end of epoch   8 | time: 4676.89s | valid loss  6.35 | valid ppl   571.52
-----------------------------------------------------------------------------------------


In [82]:
torch.save(best_model.state_dict(),'best_model_3bigx3_corrected.pt')
torch.save(model.state_dict(),'training_model_3bigx3_corrected.pt')

# Data Generation

In [224]:
import time
def generator(model: nn.Module, gen_data: Tensor, no_words = 10):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        print('i:', i)
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        else:
            src_mask_ = src_mask[:,:]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.argmax(output_softmax_permuted, dim=2)
        #print(indices[0],indices[1])
        #for j in range(batch_size):
        print('next word: ', [vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        print(i,"Gen_data: ",gen_data,"Pred_data: ",indices)
        pred_text.append([vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        if(batch_size < 16):
            gen_data = torch.cat((gen_data[:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
            
    return pred_text



In [486]:
# import time
# def nonnaive_generator(model: nn.Module, gen_data: Tensor, no_words = 10):
#     model.eval()
#     temp_text = text
#     src_mask = generate_square_subsequent_mask(bptt).to(device)
#     pred_text = []
#     for i in range(no_words):
#         print('i:', i)
#         batch_size = gen_data.size(0)
#         if batch_size != bptt:
#             src_mask_ = src_mask[:batch_size, :batch_size]
#         output_softmax = model(gen_data, src_mask_)
#         output_softmax_permuted = output_softmax.permute(1, 0, 2)
#         #print(softmax(output_softmax_permuted))
#         indices = torch.topk(output_softmax_permuted,10 ,dim=2,sorted=True).indices.squeeze(0)
#         values = torch.topk(softmax(output_softmax_permuted),10 ,dim=2,sorted = True).values.squeeze(0)
#         values = values/torch.sum(values,dim = 1,keepdims = True)
#         values = torch.flip(values,dims = (1,))
#         #print(output_softmax_permuted[indices])
#         print(indices,values)
#         ind_sampled = torch.distributions.Categorical(values).sample()
# #         index = indices.squeeze(0)[ind_sampled.unsqueeze(0)]
#         print('is',ind_sampled)
#         next_index = indices[-1][ind_sampled[-1]]
#         print(indices[-1][ind_sampled[-1]])
        
# #     return indices
        
#         #print(indices[0],indices[1])
#         #for j in range(batch_size):
        
#         print('next word: ', [vocab.lookup_token(next_index)],'values: ',values.squeeze(0)[-1])
                                  
# #         print(i,"Gen_data: ",gen_data,"Pred_data: ",indices)


#         pred_text.append([vocab.lookup_token((next_index))][0])
#         if(batch_size <= 10):
#             gen_data = torch.cat((gen_data[:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
#             batch_size= gen_data.size(0)
#         else:
#             gen_data = torch.cat((gen_data[1:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
#             batch_size= gen_data.size(0)
            
#     return pred_text



def nonnaive_generator(model: nn.Module, gen_data: Tensor, no_words = 5,k=50):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        print('i:', i)
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.topk(output_softmax_permuted,k ,dim=2).indices.squeeze(0)
        
        values = torch.topk(softmax(output_softmax_permuted),k ,dim=2).values
        values = values/torch.sum(values,dim = 2,keepdims = True)
#         values = softmax(values)
        
#         values = torch.flip(values,dims = (2,))

        
        ind_sampled = torch.distributions.Categorical(values.squeeze(0)).sample()
        next_index = indices[-1][ind_sampled[-1]]
        
        print('next word: ', vocab.lookup_token(next_index))

        print(i,"Values: ",values.squeeze(0)[-1],"Gen_data: ",gen_data,"possible tokens: ",indices[-1],"Pred_data: ",next_index)
        pred_text.append([vocab.lookup_token(next_index)][0])
        if(batch_size < 15):
            gen_data = torch.cat((gen_data[:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
            
    return pred_text

In [225]:
#Load saved model

model.load_state_dict(torch.load('best_model_3bigx3_corrected.pt'))
model.to(device)
best_model = model

In [85]:

print(sample_data[:,-1].unsqueeze(1))
print(sample_data)
z = generator(best_model, sample_data[:,-1].unsqueeze(1),no_words = 50)

tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357]], device='cuda:0')
tensor([[ 2086,   357],
        [ 5694,   465],
        [  568,   410],
        [    0,  6548],
        [  897,   293],
        [28361,     0],
        [    0,   357]], device='cuda:0')
i: 0
next word:  राज्य
0 Gen_data:  tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357]], device='cuda:0') Pred_data:  tensor([[   53,   665,   387, 10546,     3,     0,   465]], device='cuda:0')
i: 1
next word:  अमेरिका
1 Gen_data:  tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357],
        [ 465]], device='cuda:0') Pred_data:  tensor([[   53,   665,   387, 10546,     3,     0,   465,   665]],
       device='cuda:0')
i: 2
next word:  ?
2 Gen_data:  tensor([[ 357],
        [ 465],
        [ 410],
        [6548],
        [ 293],
        [   0],
        [ 357]

next word:  जापान
33 Gen_data:  tensor([[ 1803],
        [    2],
        [14639],
        [    2],
        [ 1803],
        [    2],
        [ 1803],
        [    2],
        [ 1803],
        [    2],
        [14639],
        [    2],
        [ 1803],
        [    2],
        [14639],
        [    2]], device='cuda:0') Pred_data:  tensor([[    2,   665,     2,  1803,     2,  1803,     2, 14639,     2, 14639,
             2,  1803,     2, 14639,     2,  1803]], device='cuda:0')
i: 34
next word:  ?
34 Gen_data:  tensor([[    2],
        [14639],
        [    2],
        [ 1803],
        [    2],
        [ 1803],
        [    2],
        [ 1803],
        [    2],
        [14639],
        [    2],
        [ 1803],
        [    2],
        [14639],
        [    2],
        [ 1803]], device='cuda:0') Pred_data:  tensor([[    0,     2,   666,     2, 14639,     2, 14639,     2, 14639,     2,
          1803,     2, 14639,     2,  1803,     2]], device='cuda:0')
i: 35
next word:  भियतनाम
35 Gen

In [86]:
'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त ' +' '.join(z)

'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य अमेरिका ? चीन ? जापान ? जापान ? जापान ? भियतनाम ? जापान ? भियतनाम ? जापान ? भियतनाम ? जापान ? जापान ? जापान ? भियतनाम ? जापान ? भियतनाम ? जापान ? भियतनाम ? जापान ? जापान ? जापान ? भियतनाम ? जापान ? भियतनाम ? जापान'

In [87]:
evaluate(best_model, sample_data)

8.372248649597168

In [88]:
'''
आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , 
संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त

विकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय
'''

'\nआधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , \nसंयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त\n\nविकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय\n'

In [443]:
st = ['गर्ने']
st_i = data_process(st)

In [403]:
st_i = st_i.unsqueeze(1).to(device)

In [404]:
z_ = generator(best_model, st_i,no_words =50 )

i: 0
next word:  निर्णय
0 Gen_data:  tensor([[13]], device='cuda:0') Pred_data:  tensor([[163]], device='cuda:0')
i: 1
next word:  गरेको
1 Gen_data:  tensor([[ 13],
        [163]], device='cuda:0') Pred_data:  tensor([[163,  11]], device='cuda:0')
i: 2
next word:  छ
2 Gen_data:  tensor([[ 13],
        [163],
        [ 11]], device='cuda:0') Pred_data:  tensor([[163,  11,   4]], device='cuda:0')
i: 3
next word:  ।
3 Gen_data:  tensor([[ 13],
        [163],
        [ 11],
        [  4]], device='cuda:0') Pred_data:  tensor([[163,  11,   4,   1]], device='cuda:0')
i: 4
next word:  नेपाल
4 Gen_data:  tensor([[ 13],
        [163],
        [ 11],
        [  4],
        [  1]], device='cuda:0') Pred_data:  tensor([[163,  11,   4,   1,  22]], device='cuda:0')
i: 5
next word:  राष्ट्र
5 Gen_data:  tensor([[ 13],
        [163],
        [ 11],
        [  4],
        [  1],
        [ 22]], device='cuda:0') Pred_data:  tensor([[163,  11,   4,   1,  22, 337]], device='cuda:0')
i: 6
next word:  बैंकल

next word:  निष्काशन
36 Gen_data:  tensor([[ 126],
        [ 264],
        [  69],
        [  21],
        [   4],
        [   1],
        [ 513],
        [ 271],
        [  18],
        [ 126],
        [ 264],
        [  69],
        [ 264],
        [  44],
        [1227],
        [ 678]], device='cuda:0') Pred_data:  tensor([[ 264,   69,  156,    4,    1,   89,  271,   18,  270,  264,   69,  264,
           44, 1227,  678, 5228]], device='cuda:0')
i: 37
next word:  गरेको
37 Gen_data:  tensor([[ 264],
        [  69],
        [  21],
        [   4],
        [   1],
        [ 513],
        [ 271],
        [  18],
        [ 126],
        [ 264],
        [  69],
        [ 264],
        [  44],
        [1227],
        [ 678],
        [5228]], device='cuda:0') Pred_data:  tensor([[  44,  156,    4,    1,   89,  271,   18,  270,  264,   69,  264,   44,
         1227,  678, 5228,   11]], device='cuda:0')
i: 38
next word:  छ
38 Gen_data:  tensor([[  69],
        [  21],
        [   4],
       

In [405]:
' '.join(z_)

'निर्णय गरेको छ । नेपाल राष्ट्र बैंकले आगामी आर्थिक वर्षको मौद्रिक नीतिमा लघुवित्त वित्तीय संस्था लिमिटेडको खुद मुनाफा रु एक करोड ५० लाख रहेको छ । बैंकले रु एक करोड ५० लाख ५० हजार बराबरको शेयर निष्काशन गरेको छ । बैंकले रु एक अर्ब ५० करोड ५० लाख ५० हजार'

In [487]:
st = ['म भारत भ्रमण गर्न']
st_i = data_process(st)

In [488]:
st_i = st_i.unsqueeze(1).to(device)
st_i

tensor([[ 83],
        [301],
        [700],
        [ 10]], device='cuda:0')

In [489]:
z_ = generator(best_model, st_i,no_words = 50)

i: 0
next word:  चाहन्छु
0 Gen_data:  tensor([[ 83],
        [301],
        [700],
        [ 10]], device='cuda:0') Pred_data:  tensor([[  17, 7752,   10, 2067]], device='cuda:0')
i: 1
next word:  ।
1 Gen_data:  tensor([[  83],
        [ 301],
        [ 700],
        [  10],
        [2067]], device='cuda:0') Pred_data:  tensor([[  17, 7752,   10, 2067,    1]], device='cuda:0')
i: 2
next word:  तर
2 Gen_data:  tensor([[  83],
        [ 301],
        [ 700],
        [  10],
        [2067],
        [   1]], device='cuda:0') Pred_data:  tensor([[  17, 7752,   10, 2067,    1,   19]], device='cuda:0')
i: 3
next word:  ?
3 Gen_data:  tensor([[  83],
        [ 301],
        [ 700],
        [  10],
        [2067],
        [   1],
        [  19]], device='cuda:0') Pred_data:  tensor([[  17, 7752,   10, 2067,    1,   19,    2]], device='cuda:0')
i: 4
next word:  नेपालको
4 Gen_data:  tensor([[  83],
        [ 301],
        [ 700],
        [  10],
        [2067],
        [   1],
        [  19],
   

next word:  छ
34 Gen_data:  tensor([[ 337],
        [ 513],
        [2635],
        [3851],
        [2816],
        [ 589],
        [ 478],
        [4752],
        [4649],
        [4886],
        [ 271],
        [  18],
        [ 126],
        [ 264],
        [  69],
        [  21]], device='cuda:0') Pred_data:  tensor([[ 513, 1898, 3851,  757,  589, 1200,    3, 4649, 4886,  271,   18,  126,
          264,   69,  264,    4]], device='cuda:0')
i: 35
next word:  ।
35 Gen_data:  tensor([[ 513],
        [2635],
        [3851],
        [2816],
        [ 589],
        [ 478],
        [4752],
        [4649],
        [4886],
        [ 271],
        [  18],
        [ 126],
        [ 264],
        [  69],
        [  21],
        [   4]], device='cuda:0') Pred_data:  tensor([[1898, 3851,  210,  589, 1200,    3, 4649, 4886,  271,   18,  126,  264,
           69,  264,    4,    1]], device='cuda:0')
i: 36
next word:  बैंकले
36 Gen_data:  tensor([[2635],
        [3851],
        [2816],
        [ 589

In [490]:
' '.join(st)+' ' +' '.join(z_)

'म भारत भ्रमण गर्न चाहन्छु । तर ? नेपालको पहिलो ठूलो चुनौती भनेको नेपालको आर्थिक विकासमा ठूलो टेवा पुगेको छ । नेपाल राष्ट्र बैंकले मौद्रिक नीतिमा लघुवित्त वित्तीय संस्था लिमिटेडको खुद मुनाफा रु एक करोड ५० लाख रहेको छ । बैंकले रु एक करोड ५० लाख ५० हजार बराबरको शेयर निष्काशन गरेको छ ।'

In [491]:
 x = torch.arange(1., 6.)
j = torch.topk(x, 3)

In [492]:
j.values

tensor([5., 4., 3.])

In [493]:
vocab['?']

2

In [502]:
z__ = nonnaive_generator(best_model, st_i,no_words = 100,k=50)

i: 0
next word:  पुगेँ
0 Values:  tensor([0.3439, 0.0863, 0.0536, 0.0429, 0.0387, 0.0298, 0.0277, 0.0277, 0.0180,
        0.0177, 0.0165, 0.0141, 0.0130, 0.0129, 0.0127, 0.0116, 0.0109, 0.0105,
        0.0101, 0.0097, 0.0094, 0.0088, 0.0088, 0.0087, 0.0084, 0.0082, 0.0081,
        0.0081, 0.0076, 0.0070, 0.0067, 0.0062, 0.0061, 0.0061, 0.0060, 0.0059,
        0.0057, 0.0057, 0.0056, 0.0056, 0.0054, 0.0052, 0.0051, 0.0050, 0.0050,
        0.0049, 0.0047, 0.0045, 0.0045, 0.0044], device='cuda:0',
       grad_fn=<SelectBackward0>) Gen_data:  tensor([[ 83],
        [301],
        [700],
        [ 10]], device='cuda:0') possible tokens:  tensor([ 2067,  7752,  4235,   228,   831, 18871,   479,  3055,  2680,   215,
         9108,  8665,  3077,  9440,     3,  7966,     5,   239,  2084,    17,
         7677,  4713,   474, 17893,  6605, 10854,  3493, 15250,     0,   903,
         1591, 16053, 11937, 50041,   977,  1696,   162,   205,    66,  2214,
        18855, 12762,  4198,  2699,    22,   69

       grad_fn=<SelectBackward0>) Gen_data:  tensor([[  301],
        [  700],
        [   10],
        [15250],
        [    1],
        [   18],
        [ 1360],
        [ 1002],
        [ 8561],
        [   37],
        [ 1647],
        [    1],
        [  143],
        [   46],
        [   95]], device='cuda:0') possible tokens:  tensor([  134,   125,    64,  1233,  1268,    51,   420,  2334,   976,   168,
          753,  3611,   661,  1292,  1582,  1546,   635,  2269,   601,   703,
         2219,   450,   251,  3253,  2785,   804,   254,  3720,   854,   394,
          364,  1677,  1283,   213, 10407,  4846,  1833,   215,  1979,  2905,
         2946,  1805,   415,  6117,  1389,  8240,   408,   483,  1859,  1318],
       device='cuda:0') Pred_data:  tensor(125, device='cuda:0')
i: 13
next word:  लगानीका
13 Values:  tensor([0.1904, 0.0752, 0.0605, 0.0454, 0.0412, 0.0379, 0.0299, 0.0283, 0.0245,
        0.0233, 0.0220, 0.0206, 0.0197, 0.0184, 0.0180, 0.0160, 0.0154, 0.0142,
        0.

       device='cuda:0') Pred_data:  tensor(880, device='cuda:0')
i: 25
next word:  बनाएको
25 Values:  tensor([0.1157, 0.0715, 0.0686, 0.0542, 0.0511, 0.0388, 0.0375, 0.0340, 0.0332,
        0.0260, 0.0253, 0.0223, 0.0212, 0.0209, 0.0194, 0.0158, 0.0158, 0.0155,
        0.0142, 0.0141, 0.0141, 0.0130, 0.0128, 0.0120, 0.0119, 0.0118, 0.0115,
        0.0115, 0.0109, 0.0106, 0.0106, 0.0105, 0.0102, 0.0100, 0.0100, 0.0093,
        0.0092, 0.0092, 0.0086, 0.0085, 0.0075, 0.0074, 0.0073, 0.0070, 0.0069,
        0.0067, 0.0066, 0.0065, 0.0065, 0.0064], device='cuda:0',
       grad_fn=<SelectBackward0>) Gen_data:  tensor([[   46],
        [   95],
        [  125],
        [ 3976],
        [    9],
        [ 2140],
        [    6],
        [  279],
        [   14],
        [ 3270],
        [14896],
        [    9],
        [  104],
        [  220],
        [  880]], device='cuda:0') possible tokens:  tensor([ 159, 1897,  670,   65,  284,  271,  118,  469,   17,  334,  757,  454,
         264,   

next word:  अघि
36 Values:  tensor([0.2706, 0.1170, 0.1041, 0.0581, 0.0556, 0.0492, 0.0433, 0.0411, 0.0216,
        0.0167, 0.0165, 0.0154, 0.0148, 0.0127, 0.0126, 0.0125, 0.0088, 0.0086,
        0.0084, 0.0080, 0.0073, 0.0070, 0.0058, 0.0053, 0.0052, 0.0046, 0.0045,
        0.0045, 0.0042, 0.0040, 0.0040, 0.0040, 0.0036, 0.0036, 0.0031, 0.0030,
        0.0030, 0.0028, 0.0027, 0.0026, 0.0025, 0.0022, 0.0021, 0.0020, 0.0018,
        0.0018, 0.0018, 0.0018, 0.0017, 0.0017], device='cuda:0',
       grad_fn=<SelectBackward0>) Gen_data:  tensor([[   9],
        [ 104],
        [ 220],
        [ 880],
        [ 469],
        [ 153],
        [ 107],
        [  54],
        [ 131],
        [  44],
        [  94],
        [  75],
        [ 264],
        [2703],
        [  26]], device='cuda:0') possible tokens:  tensor([  139,   253,   146,  1396,   338,   225,   213,  2332,     6,   136,
          514,    13,     5,   359,   378,    11,   339,  2250,   439,  3613,
            3,   433,  2616, 

        [ 264]], device='cuda:0') possible tokens:  tensor([  135,   149,   888,  2703,   128,  1505,   563,  3083,  1828,  3030,
         7228,     3,  1992,  1080, 23615, 29232,  8297,    31,     2,  5875,
         9426,   156, 30761,  1993,  3841,  2180,   933,  2733,    44,  4392,
          747,  3740,  6282,  4475,  1117,  2424,    95,  1776,  6887,  7252,
          950,   295,  2049,    32, 24080, 23617,  2624,   257,    64,   362],
       device='cuda:0') Pred_data:  tensor(128, device='cuda:0')
i: 49
next word:  परीक्षार्थीले
49 Values:  tensor([0.1156, 0.1007, 0.0531, 0.0507, 0.0490, 0.0388, 0.0273, 0.0268, 0.0264,
        0.0255, 0.0226, 0.0215, 0.0198, 0.0192, 0.0185, 0.0182, 0.0168, 0.0164,
        0.0160, 0.0160, 0.0155, 0.0152, 0.0142, 0.0140, 0.0139, 0.0137, 0.0128,
        0.0119, 0.0117, 0.0115, 0.0106, 0.0104, 0.0103, 0.0098, 0.0095, 0.0092,
        0.0087, 0.0086, 0.0080, 0.0079, 0.0077, 0.0077, 0.0076, 0.0076, 0.0074,
        0.0073, 0.0073, 0.0070, 0.0070, 0.0070],

next word:  उत्तिर्ण
59 Values:  tensor([0.2198, 0.1323, 0.0963, 0.0753, 0.0583, 0.0556, 0.0494, 0.0447, 0.0352,
        0.0237, 0.0206, 0.0133, 0.0116, 0.0116, 0.0115, 0.0098, 0.0096, 0.0067,
        0.0061, 0.0058, 0.0053, 0.0053, 0.0052, 0.0048, 0.0048, 0.0046, 0.0045,
        0.0044, 0.0042, 0.0041, 0.0037, 0.0036, 0.0034, 0.0034, 0.0032, 0.0031,
        0.0031, 0.0030, 0.0030, 0.0029, 0.0028, 0.0026, 0.0025, 0.0024, 0.0022,
        0.0022, 0.0022, 0.0022, 0.0021, 0.0020], device='cuda:0',
       grad_fn=<SelectBackward0>) Gen_data:  tensor([[   44],
        [   58],
        [   75],
        [  264],
        [  128],
        [16277],
        [ 2173],
        [ 3717],
        [    2],
        [  176],
        [   44],
        [  208],
        [   75],
        [  953],
        [10626]], device='cuda:0') possible tokens:  tensor([  363,  4103,     2,     7, 12087,     3,    21,    14, 20498,    88,
         1129,   316,    43,    24, 10626,  1177,  8809,  4564,   140,  8018,
         

        [    2]], device='cuda:0') possible tokens:  tensor([   62,  2833,   104,  6944,  1065,     3,    58, 11431,   132,   582,
           22, 16649, 11639,  2717, 53326,    76,  3210,   388,  3726,  9269,
         2465,  1818,  2840,  3964,   386,  2244,  2698,  8505,   229,   511,
         9175,   242,   372,  1897,    96, 50715,   253,   378,    94,   131,
          733,  1007,  3152, 43619,     0,     2, 16852,    18,  2985,   767],
       device='cuda:0') Pred_data:  tensor(2833, device='cuda:0')
i: 72
next word:  हजार
72 Values:  tensor([9.0778e-01, 3.6026e-02, 1.8549e-02, 5.3307e-03, 3.8727e-03, 3.6225e-03,
        2.9008e-03, 2.5784e-03, 2.3370e-03, 2.1127e-03, 1.4284e-03, 1.0992e-03,
        1.0253e-03, 1.0160e-03, 9.5301e-04, 7.7461e-04, 7.1626e-04, 6.4723e-04,
        6.2015e-04, 4.2140e-04, 4.0895e-04, 3.5043e-04, 3.4011e-04, 3.2331e-04,
        3.0753e-04, 2.9130e-04, 2.8882e-04, 2.8257e-04, 2.7846e-04, 2.4091e-04,
        2.2236e-04, 2.1627e-04, 2.1280e-04, 1.9272e-04,

84 Values:  tensor([0.0689, 0.0643, 0.0517, 0.0513, 0.0331, 0.0292, 0.0276, 0.0264, 0.0259,
        0.0255, 0.0253, 0.0244, 0.0233, 0.0225, 0.0218, 0.0193, 0.0191, 0.0187,
        0.0181, 0.0180, 0.0172, 0.0166, 0.0162, 0.0156, 0.0147, 0.0146, 0.0146,
        0.0145, 0.0145, 0.0143, 0.0138, 0.0136, 0.0135, 0.0133, 0.0129, 0.0125,
        0.0119, 0.0117, 0.0116, 0.0115, 0.0114, 0.0111, 0.0111, 0.0110, 0.0109,
        0.0104, 0.0104, 0.0102, 0.0101, 0.0098], device='cuda:0',
       grad_fn=<SelectBackward0>) Gen_data:  tensor([[4431],
        [   2],
        [2833],
        [  44],
        [  76],
        [  75],
        [1324],
        [   2],
        [2833],
        [  44],
        [3152],
        [   3],
        [4426],
        [ 176],
        [ 974]], device='cuda:0') possible tokens:  tensor([  94, 2666,   76, 2833, 4430, 2698, 4473,  176, 2717, 4215,   96, 2465,
         208, 3999, 3152, 2370, 2840, 2978, 1818,  953,  767, 4135,   58, 1374,
        4663,  461, 4399, 2736, 4431, 188

next word:  त्यस्तै
97 Values:  tensor([0.0805, 0.0744, 0.0670, 0.0639, 0.0536, 0.0395, 0.0355, 0.0336, 0.0332,
        0.0284, 0.0243, 0.0233, 0.0231, 0.0222, 0.0219, 0.0207, 0.0205, 0.0200,
        0.0200, 0.0199, 0.0175, 0.0150, 0.0150, 0.0148, 0.0144, 0.0143, 0.0141,
        0.0140, 0.0128, 0.0120, 0.0105, 0.0105, 0.0093, 0.0087, 0.0087, 0.0067,
        0.0067, 0.0066, 0.0065, 0.0063, 0.0061, 0.0058, 0.0057, 0.0048, 0.0048,
        0.0047, 0.0046, 0.0046, 0.0046, 0.0044], device='cuda:0',
       grad_fn=<SelectBackward0>) Gen_data:  tensor([[ 176],
        [ 974],
        [4215],
        [ 124],
        [  14],
        [ 229],
        [ 168],
        [  81],
        [4132],
        [ 461],
        [ 974],
        [2717],
        [ 124],
        [  16],
        [   1]], device='cuda:0') possible tokens:  tensor([ 171,  733, 1006, 1897,  193,  934, 2635, 5041,   22, 1525,  248,   15,
        2341,  110, 1779,  153,   19,  236,  511,   81,   18, 3405,  120,   87,
          48,  286, 1

In [503]:
' '.join(st)+' ' +' '.join(z__)

'म भारत भ्रमण गर्न पुगेँ । एक किसिमको सम्पत्ति शुद्धीकरण हुन पुग्यो । हामीले सबै भन्दा ठूलो लगानीका लागि चाहिने भएको कृषि तथा गरिबी निवारणका लागि राष्ट्रिय योजना आयोगले बनाएको सो कार्यक्रम अहिले ४ हजार ५ सय ५० रोपनी काम अघि बढाइएको उनको भनाई छ । त्यस्तै ३ हजार १ सय ५० जना परीक्षार्थीले सी प्लस ? ७ हजार ८ सय ३५ परीक्षार्थी उत्तिर्ण गरेका छन् । त्यस्तै ? ३९ हजार ३ सय ८१ ? ६२ हजार २ सय ३३ ? ६२ हजार ६७ र वित्त ७ दशमलव ६९ प्रतिशत तथा दोस्रो कम आर्थिक वृद्धिदर १७ दशमलव ६३ प्रतिशत थियो । त्यस्तै ? मौद्रिक'

In [504]:
vocab['।']

1

In [505]:
z__

['पुगेँ',
 '।',
 'एक',
 'किसिमको',
 'सम्पत्ति',
 'शुद्धीकरण',
 'हुन',
 'पुग्यो',
 '।',
 'हामीले',
 'सबै',
 'भन्दा',
 'ठूलो',
 'लगानीका',
 'लागि',
 'चाहिने',
 'भएको',
 'कृषि',
 'तथा',
 'गरिबी',
 'निवारणका',
 'लागि',
 'राष्ट्रिय',
 'योजना',
 'आयोगले',
 'बनाएको',
 'सो',
 'कार्यक्रम',
 'अहिले',
 '४',
 'हजार',
 '५',
 'सय',
 '५०',
 'रोपनी',
 'काम',
 'अघि',
 'बढाइएको',
 'उनको',
 'भनाई',
 'छ',
 '।',
 'त्यस्तै',
 '३',
 'हजार',
 '१',
 'सय',
 '५०',
 'जना',
 'परीक्षार्थीले',
 'सी',
 'प्लस',
 '?',
 '७',
 'हजार',
 '८',
 'सय',
 '३५',
 'परीक्षार्थी',
 'उत्तिर्ण',
 'गरेका',
 'छन्',
 '।',
 'त्यस्तै',
 '?',
 '३९',
 'हजार',
 '३',
 'सय',
 '८१',
 '?',
 '६२',
 'हजार',
 '२',
 'सय',
 '३३',
 '?',
 '६२',
 'हजार',
 '६७',
 'र',
 'वित्त',
 '७',
 'दशमलव',
 '६९',
 'प्रतिशत',
 'तथा',
 'दोस्रो',
 'कम',
 'आर्थिक',
 'वृद्धिदर',
 '१७',
 'दशमलव',
 '६३',
 'प्रतिशत',
 'थियो',
 '।',
 'त्यस्तै',
 '?',
 'मौद्रिक']